In [14]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from helper import *
pd.options.display.max_columns = None

## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [4]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
titanic_path = "data/"
# need one more
# what a brilliant idea to name files with space

In [5]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum_noise

In [6]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,66957,74432,96087,103120,64272,150633,181787,180349,216912,304071,1434819,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [9]:
# Remove 'Instance' as it simply represents the row number
sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = [sumdata_noise['Noisy Target']]

# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise['Noisy Target Class']

# Extract rest columns as explananatory variables
sumdata_noise_X = sumdata_noise.iloc[:, 0:-2]

# Apply Feature Scaling to the dataset 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY = StandardScaler()
sumdata_noise_X = pd.DataFrame(scX.fit_transform(sumdata_noise_X), index=sumdata_noise_X.index, columns=sumdata_noise_X.columns)
sumdata_noise_reg_Y = pd.DataFrame(scY.fit_transform(sumdata_noise_reg_Y), index=sumdata_noise_reg_Y.index, columns=sumdata_noise_reg_Y.columns)

sumdata_noise_X.head(n=1)
sumdata_noise_reg_Y(n=1)

AttributeError: 'list' object has no attribute 'columns'

## Load datasets sumdata

In [ ]:
sumdata = pd.read_csv(sumdata_path, delimiter=";")
sumdata.head(n=1)

## Preprocess sumdata dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest of the columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [ ]:
# Remove 'Instance' as it simply represents the row number
sumdata.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_reg_Y = [sumdata['Target']]

# Extract 'Nosiy Target Class' as classification target
sumdata_classif_Y = sumdata['Target Class']

# Extract rest columns as explananatory variables
sumdata_X = sumdata.iloc[:, 0:-2]

# Apply Feature Scaling to the dataset 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY = StandardScaler()
sumdata_X = pd.DataFrame(scX.fit_transform(sumdata_X), index=sumdata_X.index, columns=sumdata_X.columns)
sumdata_reg_Y = pd.DataFrame(scY.fit_transform(sumdata_reg_Y), index=sumdata_reg_Y.index, columns=sumdata_reg_Y.columns)

sumdata_X.head(n=1)

## Load housing price dataset

In [ ]:
housing_price = pd.read_csv(housing_price_path)
housing_price.head(n=5)

## Preprocess housing price dataset

- Remove 'Id' as it simply represents the row number
- Extract 'SalePrice' as regression target
- Extract 'SaleCondition' as classification target
- For explananatory variables, use all the numerical variable and use the following categorical variables
    - Neighborhood, as the career of the neighbour impacts on factors such as the consuming power of the area 
    - HouseStyle, well-designed house is more expensive
    - Foundation
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [ ]:
# Remove 'Instance' as it simply represents the row number
housing_price.drop('Id', axis = 1)

# Extract 'Nosiy Target' as regression target
housing_price_reg_Y = [housing_price['SalePrice']]

# Extract 'Nosiy Target Class' as classification target
housing_price_classif_Y = housing_price['SaleCondition']

# Extract explananatory variables
housing_price_X = housing_price.iloc[:, 0:-2]._get_numeric_data()
# housing_price_X['Neighborhood'] = pd.get_dummies(housing_price, columns=['Neighborhood'])
# housing_price_X

# # Apply Feature Scaling to the dataset 
# from sklearn.preprocessing import StandardScaler
# scX = StandardScaler()
# scY = StandardScaler()
# housing_price_X = scX.fit_transform(housing_price_X)
# housing_price_reg_Y = scY.fit_transform(housing_price_reg_Y)
# housing_price_classif_Y = housing_price_classif_Y.values

# housing_price_classif_Y

# Didn't do cross validation

In [ ]:
sumdata_noise.shape

In [ ]:
def model( X, y,  data_set, algorithm=LinearRegression): 
    
    algo_name = algorithm.__name__
    
    for chunk in data_chunks:

        if chunk > sumdata_noise.shape[0]: # if chunk is greater than the no. of examples
            break

        X = X.head(n = chunk)
        y = y.head(n = chunk)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        lm = algorithm()
        model = lm.fit(X_train, y_train)
        predictions = lm.predict(X_test)

        #print(predictions)
        
        error = mean_squared_error(y_test, predictions)**0.5  # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        print ("Data set: {} Algorithm: {}  Chunk Size: {} Error: {}".format(data_set,
                                                                          algo_name,
                                                                          chunk,
                                                                           error
                                                                         ))
              
         
    

In [ ]:
model(sumdata_noise_X, sumdata_noise_reg_Y,"Noisy Sum", LinearRegression )

In [ ]:
model(sumdata_X, sumdata_y,"Sum without noise", LinearRegression )

# DID do cross validation

In [11]:
def kFoldModel (X, y, data_set, kfolds, algorithm=LinearRegression):
    
    algo_name = algorithm.__name__
    
    kf = KFold(n_splits=kfolds)
    
    for train, test in kf.split(data_set):
        print("%s %s" % (train, test))

In [15]:
kFoldModel(sumdata_noise_X, sumdata_noise_reg_Y, "Noisy Sum", 2, LinearRegression )

[5 6 7 8] [0 1 2 3 4]
[0 1 2 3 4] [5 6 7 8]
